In [3]:
import os
import multiprocessing
import time
from shape_classes import *
from useful import *
import bisect
from sklearn.cluster import KMeans

fourier_threshold = 0.5

if __name__ == '__main__':

    start = time.time()
    pid_dict = {}
    pid_dict['main'] = os.getpid()
    program_done_event = multiprocessing.Event()
    resource_logger = multiprocessing.Process(target=log_resource_usage, args=(pid_dict,program_done_event))
    resource_logger.start()

    layout_name = "layout"  
    layout_path = os.path.join( os.getcwd(), 'images', layout_name)
    SEM_name = "SEM" 
    SEM_path = os.path.join( os.getcwd(), 'images', SEM_name)

    report = open('report.txt', 'w')
    report.write('Report')
   

    layout_window = No_window(layout_path)
    SEM_window = No_window(SEM_path)

    total_cells_layout = 0
    total_cells_SEM = 0

    while True:

         ############################ Cell Extraction ##############################
        current_window, last_window = layout_window.get_current_window()
        blobs = connected_components(current_window)

        cell_list_layout = []
        
        for blob in blobs:
            pkt = shape_packet()
            pkt.bbox = blob.bbox
            pkt.blob_img = blob.image
            pkt.centroid = ( int( blob.centroid[0]), int(blob.centroid[1]))
            pkt.set_shape_descriptor()
            bisect.insort( cell_list_layout, pkt)


        current_window, last_window = SEM_window.get_current_window()
        blobs = connected_components(current_window)

        cell_list_SEM = []
        
        for blob in blobs:
            pkt = shape_packet()
            pkt.bbox = blob.bbox
            pkt.blob_img = blob.image
            pkt.centroid = ( int( blob.centroid[0]), int(blob.centroid[1]))
            pkt.set_shape_descriptor()
            bisect.insort( cell_list_SEM, pkt)

        N_cells_layout = len(cell_list_layout)
        N_cells_SEM = len(cell_list_SEM)

        total_cells_layout += N_cells_layout
        total_cells_SEM += N_cells_SEM
        
        N_cells = min(N_cells_layout, N_cells_SEM)

         ############################ Cell Validation ##############################
        for pkt_layout, pkt_SEM in zip(cell_list_layout[:N_cells], cell_list_SEM[:N_cells]):
            distance =  min ( complex_l2( pkt_layout.Ga, pkt_SEM.Ga) , complex_l2( pkt_layout.Ga, pkt_SEM.Gb) ,
                             complex_l2( pkt_layout.Gb, pkt_SEM.Ga) , complex_l2( pkt_layout.Gb, pkt_SEM.Gb) )

            if distance > fourier_threshold:
                report.write(f'\nCell modification detected')       


        if last_window:
            break

    report.write(f'\nTotal cells in layout = {total_cells_layout}')
    report.write(f'\nTotal cells in SEM = {total_cells_SEM}')
    report.close()

    program_done_event.set()
    resource_logger.join()
    
    end = time.time()

    print(f"Total execution time = {end-start}")

Total execution time = 0.8196990489959717
